<a href="https://colab.research.google.com/github/XTMay/AI_OCR/blob/main/layoutlmv3_ner/Lec_10/Lec_10_11_LayoutLMv3_Param_Tuning_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LayoutLMv3 配置深度讲解：如何“看指标、动参数、得提升”

> 目标：一份可执行的 **参数调优作战手册**。当拿到一批训练/评估结果时，能迅速判断“症状→原因→怎么改”，并据此调整 **模型结构参数** 与 **训练超参数**，从小数据可跑通到企业级大规模训练可稳定收敛、可迭代优化。

## 目录

1. [参数分层与调优思路](#参数分层与调优思路)
2. [模型结构参数（LayoutLMv3Config）逐项详解](#模型结构参数LayoutLMv3Config逐项详解)
3. [微调超参数（TrainingArguments/自定义训练）逐项详解](#微调超参数TrainingArguments自定义训练逐项详解)
4. [“看曲线动参数”的实战决策表](#看曲线动参数的实战决策表)
5. [按数据规模与资源的推荐配置](#按数据规模与资源的推荐配置)
6. [中文/多语言与长文档的专项建议](#中文多语言与长文档的专项建议)
7. [训练稳定性与泛化的加固手段](#训练稳定性与泛化的加固手段)
8. [端到端参数模板（YAML/Trainer）](#端到端参数模板YAMLTrainer)
9. [三轮迭代示例：从基线到企业级可用](#三轮迭代示例从基线到企业级可用)

## 参数分层与调优思路

**A. 模型结构参数（决定“模型容量/表示能力/显存与速度”）**

* `vocab_size, hidden_size, num_hidden_layers, num_attention_heads, max_position_embeddings, type_vocab_size, classifier_dropout`
* 更改这些参数相当于“换模型/造新模型”，会影响加载预训练权重与显存需求。新手除非必要，**尽量使用官方权重原始配置**；确需更改时要明确代价与收益。

**B. 训练超参数（决定“如何把容量变成效果”）**

* `learning_rate, batch_size, weight_decay, warmup, epochs, dropout, lr_scheduler, gradient_clip 等`
* 大多数企业项目 **80% 的收益来自训练超参数和数据策略**（数据清洗/增强/均衡/后处理）。

**调优顺序**（强烈建议遵循）

1. 跑通基线 → 2) 调学习率/有效 batch → 3) 调序列长度与数据增强 → 4) 正则化（dropout/weight decay/label smoothing） → 5) 再考虑结构改动（层数/隐藏维度/头数）。


## 模型结构参数（LayoutLMv3Config）逐项详解

> 下面解释“是什么/有什么影响/何时需要改/怎么改 + 风险”。

### 1) `vocab_size`（默认 50265）

* **是什么**：词表大小，决定 embedding 矩阵维度。LayoutLMv3-base 采用 RoBERTa 的 BPE 词表（英文友好，中文会被拆成 byte 片段也能跑，但不一定最优）。
* **影响**：改动会导致 **嵌入矩阵形状不匹配**，需要 `model.resize_token_embeddings(new_size)` 并对新增 token 随机初始化 → 早期会有轻微退化。
* **何时改**：

  * 做 **纯中文/多语任务** 且占比很高；
  * 需要加入自定义特殊 token（如“<AMT>”）。
* **怎么改**：

  * 训练自定义 tokenizer（SentencePiece/BPE）；加载预训练再 `resize_token_embeddings`；用 **较小 LR** 微调，让新增嵌入慢慢对齐。
* **风险**：破坏了与原预训练的对齐，需要更多数据/轮数收敛。

### 2) `hidden_size`（默认 768）

* **是什么**：Transformer 层的隐藏维度，决定 **单层表示能力** 与 **注意力头可分配性**。
* **影响**：增大可提升复杂模式拟合（跨版式/跨语言），但显存与时延 **线性\~平方级** 增长（注意力是 O(L²)）。
* **何时改**：

  * 数据量大（≥10w 页）且显存/算力充足；
  * 需要更强泛化（模板极多、多语多币种）。
* **怎么改**：768→1024；同时将 `num_attention_heads` 等比例增加（确保 `hidden_size % heads == 0`）。
* **风险**：小数据下容易过拟合；需要更强正则与更长 warmup。

### 3) `num_hidden_layers`（默认 12）

* **是什么**：Transformer 层数，决定 **深度容量**。
* **影响**：层数增加 → 更深表达 → 推理更慢、显存更大。
* **何时改**：

  * 版式／语言多样性很大；
  * 预训练/继续预训练（domain-adaptive pretrain）样本多。
* **怎么改**：12→16/24；建议用 **大 batch + 小 LR**；配合 **梯度检查点/ZeRO/FSDP**。
* **风险**：训练不稳/梯度消失 → 必备 warmup、梯度裁剪、学习率调小。

### 4) `num_attention_heads`（默认 12）

* **是什么**：Multi-Head Attention 的头数。
* **影响**：更多头 = 更细的子空间注意力；需与 `hidden_size` 匹配（可除尽）。
* **何时改**：跟随 `hidden_size` 调整；或做消融试验（保持 `hidden_size` 不变，改头数通常收益不大）。
* **风险**：头太多但维度太小，单头表达力下降；头太少则注意不足。

### 5) `attention_probs_dropout_prob`（默认 0.1）

* **是什么**：作用在注意力权重上的 dropout。
* **影响**：提高泛化，降低过拟合；过大将损伤对齐能力（词↔patch）。
* **经验**：0.1 起步；小数据/模板单一可调到 0.15–0.2；大数据通常 0.1 足够。

### 6) `max_position_embeddings`（默认 512）

* **是什么**：文本序列的最大长度（**对图像 patch 有单独的位置编码**，不与此共享）。
* **影响**：设太小 → token 被截断，实体跨断点；设太大 → 显存/速度开销增大。
* **何时改**：OCR token 很多（细粒度切词/密集表格），页级序列超过 512。
* **怎么改**：

  * **优先策略**：页面分块/按行合并 token/滑窗 + 重叠（例如 stride=64）；
  * **不得已**再把 `max_position_embeddings` 提到 768/1024（注意显存）。
* **风险**：长序列注意力 O(L²)；训练显著变慢。

### 7) `type_vocab_size`（默认 2）

* **是什么**：token\_type\_ids 的类型数（句子对/区域分段）。RoBERTa 家族 **通常不使用** segment embedding。
* **建议**：保持默认即可；确需区分“页眉/正文/表格”等段，可自定义，但**需要重新训练/微调**让模型学到含义。

### 8) `classifier_dropout`（默认 0.1）

* **是什么**：仅作用在分类头的 dropout。
* **影响**：可抑制分类头过拟合；对小数据/噪声标签特别有用。
* **经验**：0.1 起步；过拟合明显（训练 F1 高、验证 F1 低）时可提到 0.2–0.3。

> ⚠️ 实际还会有 `hidden_dropout_prob`（层内全连接的 dropout）与图像/patch 的数据增广，配合使用能更稳。



## 微调超参数（TrainingArguments/自定义训练）逐项详解

> 这些是日常“见招拆招”的主力武器。

### 核心三件套：学习率 / 有效批次 / 训练轮数

* **学习率 `learning_rate`**：最敏感的旋钮。

  * 小数据首选 `5e-5 / 3e-5`；中大数据 `3e-5 / 2e-5 / 1e-5`。
  * **线性缩放经验**：`LR ∝ 有效批次`（保持 loss 曲线平稳）。
* **有效批次 `global_batch = per_device_batch × #GPU × grad_accum`**

  * 结构化抽取任务常见稳定区间 **128–512**。
* **轮数 `num_train_epochs`**

  * 小数据 10–30；中等 6–12；大数据 3–8（配合早停）。

### 正则化与预热

* **weight\_decay（0.01）**：足以稳定。
* **dropout（0.1–0.3）**：过拟合时增加；欠拟合时适度减小。
* **warmup**：`warmup_ratio=0.06~0.1` 更稳；长训练或大模型可 0.1–0.2。
* **label\_smoothing**：0.05–0.1 可降低过置信，改善 O 类占比大的 NER 任务。

### 学习率调度

* `linear`（默认易用）、`cosine`（长训更平滑）、`polynomial`（后期抑制较强）。
* 验证 F1 前期波动大→适当增大 warmup/改为 `cosine`。

### 其它关键项

* **gradient\_accumulation\_steps**：扩大有效 batch 的关键。
* **gradient\_clip (1.0)**：防爆梯度的重要保险。
* **fp16/bf16**：建议开启；A100/朴素 RTX40 可优先 `bf16=True`。
* **evaluation\_strategy**：`"steps"`（小数据/频繁看曲线）或 `"epoch"`（大数据）。
* **load\_best\_model\_at\_end=True**：以 `metric_for_best_model="f1"` 做早停标准。

---

## “看曲线动参数”的实战决策表

> 观察 **训练/验证 loss、F1、学习率、显存、速度**；结合样例可视化（错在哪个字段/边界/模板）。

| 现象/症状                | 可能原因                                  | 快速动作（从上到下依次尝试）                                                                                                          |
| -------------------- | ------------------------------------- | ----------------------------------------------------------------------------------------------------------------------- |
| 训练 F1 高、验证 F1 低（过拟合） | LR 偏大；正则不足；样本少/模板单一                   | ① 降 `learning_rate`（×0.5）② 提 `classifier_dropout` 至 0.2–0.3；③ 提 `hidden_dropout_prob` 0.1→0.2；④ 加强数据增强/模板扰动；⑤ 提高 warmup |
| 训练/验证 loss 同时不降或震荡   | LR 太大；批次太小；scheduler 不稳               | ① 降 LR 30–50% ② 增大有效 batch（grad\_accum）③ 增大 warmup 比例（0.1）④ 改 `linear` → `cosine` ⑤ 开 `gradient_clip`                   |
| 验证 F1 低且“金额/币种”混淆    | 训练样本缺少列头-值配对；序列截断；后处理缺失               | ① 加入“列头+数值”的样例；② 提升 `max_length` 或滑窗合并行；③ 增加币种符号/列头增强；④ 引入后处理（邻域规则投票）                                                   |
| 实体经常被截断或跨行丢失         | `max_position_embeddings` 太小；OCR 粒度太细 | ① 合并同一行 token（连号、金额）② 滑窗 + 重叠 ③ 必要时 `max_position_embeddings` 提到 768/1024                                               |
| 小数据下训练慢、收敛晚          | 预热太少；LR 偏小                            | ① 提 `warmup_ratio` 到 0.1–0.2 ② LR→`5e-5` ③ 早期冻结前几层 3–6 层加速收敛                                                            |
| 多语言/新模板掉点            | 语料偏科；词表不适配；版式差异                       | ① 增加目标语言样本占比 ② 自训 tokenizer 或引入多语模型初始化 ③ 加强版面增广（旋转/线条/字体）                                                               |
| 显存爆                  | 序列太长；batch 过大；层太深                     | ① 降 `max_length` 或换滑窗 ② 减 per\_device\_batch + 增 grad\_accum ③ 开启梯度检查点/ZeRO/FSDP                                        |

---

## 按数据规模与资源的推荐配置

### A. 小数据（200–2k 页，单/双卡）

* `lr=5e-5`，`global_batch≈32–64`（用 grad\_accum 实现）
* `epochs=15–30`，`warmup_ratio=0.1`
* `dropout=0.2–0.3`（分类头/隐藏层）
* `max_length=512`，优先行级合并；需要时滑窗
* **技巧**：冻结底层 3–6 层先训、再全量解冻微调一轮

### B. 中等（1–10 万页，多卡）

* `lr=3e-5→2e-5`，`global_batch=128–256`
* `epochs=8–12`，`warmup_ratio=0.06–0.1`
* `dropout=0.1–0.2`
* DeepSpeed ZeRO-2，fp16/bf16，梯度检查点
* 强化数据增强（模板/语言/噪声）

### C. 大规模（10–50 万+，集群）

* `lr=2e-5→1e-5`，`global_batch=256–512`
* `epochs=3–8`，`warmup_ratio=0.03–0.06`
* FSDP/ZeRO-3，bf16，激活检查点
* **可考虑结构加大**（layers 24 / hidden 1024），注意成本与稳定性

---

## 中文/多语言与长文档的专项建议

* **中文 tokenizer**：RoBERTa 的 byte-BPE 能跑但不优。中文占比高时：

  1. 训练中文/多语专用 tokenizer；
  2. `resize_token_embeddings` 后小 LR 微调更久；
  3. 中文金额/日期做**后处理兜底**（千分位、小数点、全角/半角、负号）。
* **多语言混训**：保证各语言至少 20–30% 占比，避免偏科；币种符号/列名要覆盖：`人民币/CNY/RMB/¥`, `USD/$`, `EUR/€`。
* **长文档**：优先 **行/区域合并 + 滑窗**；必要时提高 `max_position_embeddings`，但要评估显存与速度。

---

## 训练稳定性与泛化的加固手段

* **损失函数**：

  * O 类占比过大 → **类别权重**（降低 O 权重）或 **Focal Loss**（γ=1–2）
  * **Label Smoothing** 0.05–0.1 缓解过置信
* **数据策略**：

  * **模板扰动**：列头随机同义替换（合计/价税合计/Amount Due）；
  * **图像增广**：轻微旋转、模糊、线条变形（勿过强，MIM 预训练已学结构）；
  * **主动学习**：不确定性采样 + 多样性采样回标，迭代提升长尾。
* **监控**：

  * 分字段 F1（INV\_NO/AMT\_TOTAL/AMT\_NET/CUR）；
  * 新模板/New vendor 子集；
  * 线上“人工复核率、自动通过率、回退率”。

---

## 端到端参数模板（YAML/Trainer)

```yaml
# configs/enterprise_invoice.yaml
model:
  name: microsoft/layoutlmv3-base
  max_position_embeddings: 512     # 优先滑窗与行合并，确需长序列再提
  classifier_dropout: 0.2          # 小~中数据时偏大一点更稳
train:
  learning_rate: 3.0e-5
  weight_decay: 0.01
  warmup_ratio: 0.10
  num_train_epochs: 10
  per_device_train_batch_size: 4
  per_device_eval_batch_size: 4
  gradient_accumulation_steps: 8   # 有效 batch ≈ 64（单卡示例）
  fp16: true
  gradient_checkpointing: true
  lr_scheduler_type: cosine
  logging_steps: 50
  evaluation_strategy: steps
  eval_steps: 500
  save_steps: 500
  load_best_model_at_end: true
  metric_for_best_model: f1
regularization:
  label_smoothing_factor: 0.1
  attention_probs_dropout_prob: 0.1
  hidden_dropout_prob: 0.2
data:
  max_length: 512
  window_stride: 64                 # 若启用滑窗
  bbox_norm: 1000                   # OCR bbox 归一范围
```

> 多卡/大批次时：把 `gradient_accumulation_steps` 降低，提升 `per_device_train_batch_size`；中大规模把 `lr` 降到 `2e-5/1e-5` 并把 `epochs` 缩到 `6/4`。

---

## 三轮迭代示例：从基线到企业级可用

**Round 0（基线）**

* `lr=5e-5, global_batch=32, epoch=12, dropout=0.1, max_len=512`
* 现象：训练 F1=0.95，验证 F1=0.84，字段混淆主要在 **AMT\_TOTAL vs AMT\_NET**；币种缺失率 20%。

**动作 → Round 1（稳定泛化）**

* 调参：`lr=3e-5`，`classifier_dropout=0.2`，`hidden_dropout=0.2`，`warmup_ratio=0.1`；
* 数据：加入“列头+金额”样例、币种符号覆盖；长行做合并，少量滑窗；后处理增加邻域规则（500px 内找列头/符号）。
* 结果：验证 F1=0.89；币种缺失 9%。

**动作 → Round 2（扩容&稳态）**

* 资源：多卡，`global_batch=192`（per\_device×GPU×accum）；
* 调度：`lr=2e-5, cosine, epochs=8, warmup=0.06`；
* 正则：`label_smoothing=0.1`；主动学习回收 1k 条长尾模板再训练。
* 结果：验证 F1=0.92；字段级 EM 提升到 0.90；线上人工复核率下降 35%。

---

### 经验总结

* **先调 LR 与有效批次**；
* **被截断就先滑窗，不要盲目加 max\_position\_embeddings**；
* **过拟合就加 dropout/label smoothing/数据增强**；
* **混淆字段就补“列头+数值”样本并加后处理**；
* **多语就补语料与符号覆盖，不行再自训 tokenizer**；
* **大规模才考虑加深/加宽模型**（layers/hidden/heads）。


# LayoutLMv3 发票信息抽取 - 参数调优

本 Notebook 演示如何：
1. 加载预训练的 LayoutLMv3 模型；
2. 构建最小发票样例数据集；
3. 设置 **多组参数配置** 并进行训练；
4. 自动记录训练评估指标并绘制对比图；
5. 演示如何根据结果调整参数。



In [ ]:

!pip install transformers datasets seqeval -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:

import torch
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from seqeval.metrics import f1_score, accuracy_score
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
import random, os


In [ ]:

# 构造最小玩具数据集（2 张合成发票样例）
samples = [
    {
        "id": "invoice_001",
        "words": ["Invoice", "No.", "12345", "Amount", "100", "(USD)"],
        "bboxes": [[100,200,180,220],[190,200,230,220],[240,200,300,220],
                   [100,260,180,280],[190,260,240,280],[250,260,320,280]],
        "labels": ["B-INVOICE","I-INVOICE","I-INVOICE","B-AMOUNT_TAX","I-AMOUNT_TAX","B-CURRENCY"],
        "image_path": "invoice1.png"
    },
    {
        "id": "invoice_002",
        "words": ["发票号码","67890","金额","800","(人民币)"],
        "bboxes": [[100,200,180,220],[190,200,240,220],[100,260,180,280],[190,260,240,280],[250,260,320,280]],
        "labels": ["B-INVOICE","I-INVOICE","B-AMOUNT_TAX","I-AMOUNT_TAX","B-CURRENCY"],
        "image_path": "invoice2.png"
    }
]

# 生成空白图片模拟发票背景
for s in samples:
    img = Image.new("RGB", (400,400), color=(255,255,255))
    draw = ImageDraw.Draw(img)
    for w, b in zip(s["words"], s["bboxes"]):
        draw.rectangle(b, outline="black")
        draw.text((b[0], b[1]-10), w, fill="black")
    img.save(s["image_path"])

dataset = Dataset.from_list(samples)
dataset = dataset.train_test_split(test_size=0.5, seed=42)
dataset = DatasetDict({
    "train": dataset["train"],
    "validation": dataset["test"]
})
dataset


DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'bboxes', 'labels', 'image_path'],
        num_rows: 1
    })
    validation: Dataset({
        features: ['id', 'words', 'bboxes', 'labels', 'image_path'],
        num_rows: 1
    })
})

In [ ]:
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
label_list = list(set([l for ex in samples for l in ex["labels"]]))
label2id = {l:i for i,l in enumerate(label_list)}
id2label = {i:l for l,i in label2id.items()}

def preprocess(batch):
    images = [Image.open(p).convert("RGB") for p in batch["image_path"]]
    encoded = processor(images, batch["words"], boxes=batch["bboxes"], word_labels=batch["labels"],
                        padding="max_length", truncation=True, max_length=512)
    encoded["labels"] = [[label2id[l] for l in labs] + [-100]*(512-len(labs)) for labs in batch["labels"]]
    return encoded

encoded_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)
encoded_dataset.set_format("torch")
encoded_dataset

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask', 'bbox', 'pixel_values'],
        num_rows: 1
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask', 'bbox', 'pixel_values'],
        num_rows: 1
    })
})

In [ ]:

def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=2)
    true_labels = [[id2label[l] for l in lab if l != -100] for lab in labels]
    true_preds = [[id2label[p] for (p,l) in zip(pred, lab) if l != -100] for pred, lab in zip(preds, labels)]
    return {"f1": f1_score(true_labels, true_preds), "accuracy": accuracy_score(true_labels, true_preds)}


In [ ]:
param_configs = [
    {"name":"config_A_highLR", "lr":5e-5, "dropout":0.1, "epochs":5},
    {"name":"config_B_lowLR", "lr":2e-5, "dropout":0.2, "epochs":8}
]

results = {}

for cfg in param_configs:
    print(f"Training {cfg['name']}...")
    model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                             num_labels=len(label_list),
                                                             classifier_dropout=cfg["dropout"])
    args = TrainingArguments(
        output_dir=f"./results/{cfg['name']}",
        learning_rate=cfg["lr"],
        num_train_epochs=cfg["epochs"],
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        eval_strategy="epoch",
        save_strategy="no",
        logging_steps=5,
        fp16=False,
        report_to="none"
    )
    trainer = Trainer(model=model, args=args, train_dataset=encoded_dataset["train"],
                      eval_dataset=encoded_dataset["validation"],
                      tokenizer=processor.tokenizer,
                      compute_metrics=compute_metrics)
    trainer.train()
    eval_res = trainer.evaluate()
    results[cfg["name"]] = eval_res

Training config_A_highLR...


Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-773648570.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args, train_dataset=encoded_dataset["train"],
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,1.486973,0.000000,0.200000
2,No log,1.382281,0.000000,0.400000
3,No log,1.328193,0.000000,0.400000
4,No log,1.293793,0.000000,0.400000
5,1.121400,1.272194,0.000000,0.400000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modelin

Training config_B_lowLR...


Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-773648570.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args, train_dataset=encoded_dataset["train"],
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,1.636155,0.000000,0.200000
2,No log,1.599169,0.000000,0.000000
3,No log,1.575871,0.000000,0.000000
4,No log,1.557350,0.333333,0.400000
5,1.351700,1.545164,0.000000,0.400000
6,1.351700,1.533814,0.000000,0.400000
7,1.351700,1.527117,0.000000,0.400000
8,1.351700,1.523958,0.000000,0.400000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modelin

## 1. 超参数配置

```python
param_configs = [
    {"name":"config_A_highLR", "lr":5e-5, "dropout":0.1, "epochs":5},
    {"name":"config_B_lowLR", "lr":2e-5, "dropout":0.2, "epochs":8}
]
```




### lr（学习率）
控制模型每次参数更新的步长。



*   高学习率：训练快，但可能震荡或不收敛。
*   低学习率：收敛稳定，但训练速度慢。

## dropout（丢弃率）
防止过拟合，通过随机丢弃神经元实现。

*   高 dropout：减少过拟合，但可能欠拟合。
*   低 dropout：学习能力强，但小数据集容易过拟合。

### epochs（训练轮数）
数据集完整训练的次数。

*   小数据集：可适当增加轮数。
*   大数据集：轮数不宜过多，以免过拟合。


## 2. 模型初始化

```
model = LayoutLMv3ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv3-base",
    num_labels=len(label_list),
    classifier_dropout=cfg["dropout"]
)
```

重点

*   from_pretrained：加载微软官方预训练 LayoutLMv3
*   num_labels：分类类别数（如发票关键字段）
*   classifier_dropout：只作用于分类头，可单独调整

微调策略

*   冻结 backbone：数据少时只训练分类头
*   全量训练：数据多时微调 backbone + 分类头，提高泛化能力

## 3. TrainingArguments 训练参数
```
args = TrainingArguments(
    output_dir=f"./results/{cfg['name']}",
    learning_rate=cfg["lr"],
    num_train_epochs=cfg["epochs"],
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=5,
    fp16=False,
    report_to="none"
)
```
重点参数

| 参数 | 作用 | 调整建议 |
|------|------|---------|
| output_dir | 保存训练结果 | 每个配置单独目录，便于对比 |
| learning_rate | 学习率 | 调整收敛速度和稳定性 |
| num_train_epochs | 训练轮数 | 可结合 early stopping 调整 |
| per_device_train_batch_size | 每卡 batch | 显存小可用 2~4 |
| eval_strategy | 评估策略 | "epoch" 每轮评估一次 |
| save_strategy | 保存策略 | 临时实验可设为 "no" |
| logging_steps | 日志频率 | 可调高或低 |
| fp16 | 半精度训练 | 显存受限可开启，训练加速 |
| report_to | 日志平台 | 本地训练可关闭 |



### 4. Trainer 初始化
```
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=processor.tokenizer,
    compute_metrics=compute_metrics
)
```

重点
*   train_dataset / eval_dataset：训练和验证数据
*   tokenizer：将 OCR 文本编码成 token
*   compute_metrics：计算评估指标（Precision、Recall、F1）

微调策略
*   冻结 backbone: model.base_model.requires_grad_(False)
*   全量微调: 训练 backbone + 分类头

### 5. 超参数调优建议
*   学习率 (lr)：通常 1e-5 ~ 5e-5
*   Dropout：
*   小数据集：0.2–0.3
*   大数据集：0.1–0.2
*   Epochs：
*   小数据集：5–10
*   大数据集：3–5

训练技巧
*   Early stopping：验证集 F1 不再提升时提前停止
*   梯度累积：模拟大 batch，节省显存
*   fp16：半精度训练，节省显存加速
*   逐步解冻：先训练分类头，再训练 backbone


### 6. 评估与结果对比

```
eval_res = trainer.evaluate()
results[cfg["name"]] = eval_res
```

	•	对比不同配置（如 config_A_highLR 与 config_B_lowLR）的 F1、Precision、Recall
	•	分析：
	•	高学习率：训练快但 F1 波动大
	•	低学习率 + 高 epochs：训练稳定
	•	高 dropout：防止过拟合，但可能欠拟合小数据集


In [ ]:

# 可视化不同配置的结果对比
names = list(results.keys())
f1s = [results[n]["eval_f1"] for n in names]
plt.bar(names, f1s)
plt.ylabel("F1 score")
plt.title("不同超参数配置下的 F1 对比")
plt.show()


In [ ]:

# 在真实任务中，我们会分析预测错误的 token
# 这里仅演示随机错误可视化
errors = random.sample(samples,1)
print("示例错误分析:")
for e in errors:
    print(e["words"], e["labels"])


示例错误分析:
['Invoice', 'No.', '12345', 'Amount', '100', '(USD)'] ['B-INVOICE', 'I-INVOICE', 'I-INVOICE', 'B-AMOUNT_TAX', 'I-AMOUNT_TAX', 'B-CURRENCY']


In [ ]:

# 主动学习示例（不确定性采样）
# 假设我们对验证集做预测，计算最大 softmax 概率的置信度
model.eval()
batch = encoded_dataset["validation"][0]
for k,v in batch.items():
    batch[k] = v.unsqueeze(0)
outputs = model(**batch)
probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
confidences, _ = probs.max(-1)
avg_conf = confidences[batch["labels"]!=-100].mean().item()
print(f"平均置信度: {avg_conf:.2f} → 可用于主动学习筛选低置信样本进行再标注")


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1773: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


平均置信度: 0.31 → 可用于主动学习筛选低置信样本进行再标注
